## Лекция 2  BM5    

### TfidfVectorizer

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        |  слово1  |  слово2  |  слово3  |  слово4
# текст1 |   0.6    |    0.5   |   0.6    |    0
# текст2 |   0      |    0.6   |   0.8    |    0
# текст3 |   0.9    |    0.4   |   0      |    0
# текст4 |   0      |    0     |   0      |    1
 
# чтобы получить сгенерированный словарь, из приведенной структуры TfidfVectorizer
# порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
vectorizer.vocabulary_.get('слово3') # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
new_doc = vectorizer.transform(['слово1 слово4 слово4', 'слово2 слово3 слово 4']).toarray()  # результат [[0.36673901, 0, 0, 0.93032387]]
X.dot(new_doc.transpose())

array([[0.22505553, 0.78956515],
       [0.        , 1.        ],
       [0.33994387, 0.23609724],
       [0.93032387, 0.        ]])

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [2]:
### реализуйте эту функцию ранжирования 
from math import log
import statistics
import re

k = 2.0
b = 0.75

collection = ['Они зашли в дом. В доме было пусто.',
              'Передо мной сидел рыжий котик. У него был длинный хвост.',
              'Я выглянул в окно. Мимо ехал синий автобус.',
              'Мы вышли на улицу. Что-то было явно не так.',
              'На столе стояла ваза. В ней были цветы.',
              'Надо найти заправку. Топлива почти нет.',
              'Автобус проехал под железнодорожным мостом. Наверху стоял поезд.',
              'В школе начался урок. Все сели за парты и достали свои тетрадки.']

collection_length = []
n_big = len(collection)

for c in collection:
  collection_length.append(len(c))

avgdl = statistics.mean(collection_length)

# d - один документ
# q - набор слов

def bm25(d,q) -> float:
  l_d = len(d)
  def frequency(single_q):
    tf_q_d = 0
    for word in d.split():
      word = re.sub('[!@#$%^&*\(\)\[\]\{\}.,><?/-/"]',' ',word)
      word = word.lower()
      if single_q == word:
        tf_q_d += 1
    return tf_q_d
  
  def get_idf(single_q):
    
    def number_of_docs(another_q):
      doc_num = 0
      for col in collection:
        #print(type(col))
        if another_q in col:
          doc_num += 1
      return doc_num
  
    end_result = log((n_big - number_of_docs(single_q) + 0.5)/
                     (number_of_docs(single_q) + 0.5))
    return end_result
  
  final_meanings = []

  for query in q:
    query = query.lower()
    #print(query)
    upper_half = frequency(query) * (k + 1)
    lower_half = frequency(query) + k * (1 - b + b * (l_d)/avgdl)
    final_meanings.append(get_idf(query) * upper_half/lower_half)

  return sum(final_meanings)

  
  #query_idfs = []
  #for query in q:
    #query_idfs.append(get_idf(query))
  
  #for qq in q:
    
  #def tf(element):
  #  d
  #n_qi = 0
  #for d in documents:
  #def idf(q_i):
  #  idf_qi = log((N - n_qi + 0.5)/n_qi + 0.5)
  #  return 

In [3]:
lis = ['в','было','автобус']

for coll in collection:
  print(bm25(coll,lis))

-3.6225241954868634
0.0
-2.9852014051541427
1.0067699682132274
-3.122649527405844
0.0
1.3774630624854982
-2.4248507503319128


### __Задача 1__:    
Реализуйте поиск с метрикой *TF-IDF* через умножение матрицы на вектор.
Что должно быть в реализации:
- проиндексированная база, где каждый документ представлен в виде вектора TF-IDF
- функция перевода входяшего запроса в вектор по метрике TF-IDF
- ранжирование докуменов по близости к запросу по убыванию

В качестве корпуса возьмите корпус вопросов в РПН по Covid2019. Он состоит из:
> файл **answers_base.xlsx** - база ответов, у каждого ответа есть его номер, тематика и примеры вопросов, которые могут быть заданы к этому ответу. Сейчас проиндексировать надо именно примеры вопросов в качестве документов базы. Понимаете почему?

> файл **queries_base.xlsx** - вопросы юзеров, к каждому из которых проставлен номер верного ответа из базы. Разделите эти вопросы в пропорции 70/30 на обучающую (проиндексированную как база) и тестовую (как запросы) выборки. 


In [4]:
#import openpyxl
#from pathlib import Path
import pandas as pd
import unicodedata
#from sklearn.feature_extraction.text import TfidfVectorizer

#answers = openpyxl.load_workbook('answers_base.xlsx')
#queries = openpyxl.load_workbook('queries_base.xlsx')

#answers_sheet = answers.active
#queries_sheet = queries.active

answers = pd.read_excel('answers_base.xlsx')
queries = pd.read_excel('queries_base.xlsx')
#base_1 = 

In [5]:
queries

,Текст вопроса,Номер связки\n,Тематика,Unnamed: 3,Unnamed: 4
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
...,...,...,...,...,...
2294,Добрый день.\nГражданин Украины проживает в РФ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2295,Добрый вечер. Нахожусь в командировке за грани...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2296,Здравствуйте! \nНам пришёл положительный резул...,1.0,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n",NaN,NaN
2297,"Добрый день, хотелось бы получить консультацию...",45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ\n",NaN,NaN


In [6]:
#new_seventy = queries[:int(len(queries)*0.7)]
#other_seventy = queries[1609:]

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(queries, test_size=0.3)

new_train = pd.DataFrame()
new_train['Текст вопроса'] = pd.concat([train['Текст вопроса'],answers['Текст вопросов']],ignore_index = True)
new_train['Номер связки\n'] = pd.concat([train['Номер связки\n'], answers['Номер связки']], ignore_index=True)

from sklearn.feature_extraction.text import TfidfVectorizer


X_train = vectorizer.fit_transform(new_train['Текст вопроса'].values.astype('U'))
y_train = new_train['Номер связки\n']

X_test = vectorizer.transform(test['Текст вопроса'].values.astype('U'))
y_test = test['Номер связки\n']


In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *

y_train = y_train.fillna(0)
y_test = y_test.fillna(0)

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
predicted = clf.predict(X_test)
acc = accuracy_score(y_test, predicted)
print(acc)
test['Predicted'] = predicted

0.527536231884058


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [9]:
def get_tfidf(text, vectorizer, m):
  text = text.lower()
  new_doc = vectorizer.transform([text]).toarray()
  #print(new_doc)
  new_m = m.dot(new_doc[0])
  return new_m

matrix = vectorizer.fit_transform(answers['Текст вопросов'].fillna(''))
#for g in get_tfidf('Коронавирус', vectorizer, matrix):
#  print(g)

In [10]:
#new_seventy
#print(len(get_bm25('Белки оранжевые прыгают в лесу')))
#print(len(get_tfidf('Белки оранжевые прыгают в лесу',vectorizer,matrix)))


In [11]:
#X = vectorizer.fit_transform(answers['Текст вопроса'].values.astype('U'))
#Y = vectorizer.transform(other_seventy['Текст вопроса'].values.astype('U')).toarray()



#answers['tfidf'] = get_tfidf

#for c in get_tfidf('Белки оранжевые прыгают в лесу',vectorizer,matrix):
#  print(c)
#for g in get_tfidf('Коронавирусы разные бывают',vectorizer,matrix):
#  print(g)

In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test['Номер связки\n'].fillna(0), predicted))

0.527536231884058


### __Задача 2__:    
Аналогичная задаче1 с другой метрикой 

Реализуйте поиск с метрикой *BM25* через умножение матрицы на вектор. Что должно быть в реализации:

- проиндексированная база, где каждый документ представлен в виде вектора BM25
- функция перевода входяшего запроса в вектор по метрике BM25
- ранжирование докуменов по близости к запросу по убыванию

In [13]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in answers['Текст вопросов'].fillna('')]
bm25 = BM25Okapi(tokenized_corpus)

def get_bm25(query):
  query = query.lower()
  tokenized_query = query.split(" ")
  scores = bm25.get_scores(tokenized_query)
  return scores

tf_vecs = []
bm_vecs = []

for a in answers['Текст вопросов']:
  tf_vecs.append(get_tfidf(str(a),vectorizer,matrix))
  bm_vecs.append(get_bm25(str(a)))

answers['Векторы tfidf'] = tf_vecs
answers['Векторы BM25'] = bm_vecs

In [14]:
answers

,Номер связки,Текст вопросов,Текст ответа,Тематика,Векторы tfidf,Векторы BM25
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,"[1.0, 0.028905995740077357, 0.0054801994223749...","[249.76443099199744, 19.768615742726478, 5.146..."
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ,"[0.02890599574007736, 0.9999999999999994, 0.28...","[15.606781498007713, 210.73750314990025, 39.85..."
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ,"[0.005480199422374904, 0.2811530156309264, 0.9...","[1.442809566337883, 26.82492249371232, 49.5398..."
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ,"[0.038838204231980145, 0.13721858039623075, 0....","[11.635648961094176, 10.499470402970005, 13.26..."
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ,"[0.026444332351388367, 0.08143505062467657, 0....","[3.6790036808537403, 3.881268333858796, 2.5731..."
5,40,"Что делать, тем кто находится в группе риска?\...","Лицам, входящим в группы риска, следует соблюд...",ГРУППЫ РИСКА,"[0.02694134330244139, 0.15397230006879287, 0.0...","[2.4541031616938414, 4.4311373522433675, 2.991..."
6,210,Кто в группе риска по короновирусу?\nКто наход...,Люди всех возрастов рискуют заразиться вирусом...,ГРУППЫ РИСКА,"[0.033118267906222694, 0.01265094987369082, 0....","[9.10733500340021, 5.141626690629639, 1.286575..."
7,43,Что должны обрабатывать в многоквартирных дома...,"Дезинфекция — это комплекс мероприятий, направ...",ДЕЗИНФЕКЦИЯ ОТКРЫТЫХ ПРОСТРАНСТВ И МНОГОКВАРТИ...,"[0.03877212383596491, 0.042343471127893634, 0....","[34.13691606971273, 28.194394457656905, 12.757..."
8,82,"Заканчивается виза у иностранного гражданина, ...",В связи с ситуацией по коронавирусу иностранны...,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...","[0.002879950349943883, 0.0, 0.0, 0.0, 0.0, 0.0...","[2.2248558300904153, 1.3888767165193727, 0.0, ..."
9,89,Как узнать судьбу сына/родителя/родственника (...,Мы понимаем ваше беспокойство и для получения ...,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...","[0.032025667150230225, 0.03658899196560838, 0....","[8.781261739805425, 5.884492470458618, 0.0, 0...."
